In [10]:
import pandas as pd
from langdetect import detect
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
lem = WordNetLemmatizer() #create lemmatizer

In [12]:
df = pd.read_csv('../data/processedDict.csv')

In [14]:
lemmatised_words = []
df['word'] = df['word'].apply(lambda x: lem.lemmatize(x, pos='n'))
pattern = '|'.join(df['word']) #augment dictionary to pattern form -> used for vectorised function application
pattern

'information|recommendation|tip|advice|suggestion|direction|instruction|useful|detail|info|explained|communication|question|quick|quickly|communicative|communicate|respond|responded|answer|answered|prompt|communicated|text|request|question|promptly|communicating|request|communication|answering|email|response|responding|communicator|response|concern|responds|replied|reply|query|text|inquiry|answer|check|provided|arrival|arrived|late|offered|key|hour|key|flight|prepared|reception|checking|checked|departure|waiting|arriving|timely|punctual|checkout|checkin|punctuality|helpful|welcoming|available|accommodating|welcome|responsive|help|welcomed|need|helped|flexible|hospitality|attentive|incredibly|greeted|ready|hospitable|organized|generous|availability|warmly|considerate|caring|precise|helping|uncomplicated|reliable|impeccable|respectful|accomodating|arrangement|organised|generosity|responsible|discrete|location|from|close|quiet|area|walk|located|restaurant|city|neighborhood|get|away|walkin

In [15]:
filepath = '../data/en_reviews/london_en.csv'
reviews = pd.read_csv(filepath)
#del reviews['Unnamed: 0']
comments_original = reviews['comments'] #to add back before saving to csv
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments,langs
0,11551,30672,2010-03-21,93896,Shar-Lyn,"The flat was bright, comfortable and clean and...",en
1,11551,32236,2010-03-29,97890,Zane,We stayed with Adriano and Valerio for a week ...,en
2,11551,41044,2010-05-09,104133,Chase,Adriano was a fantastic host. We felt very at ...,en
3,11551,48926,2010-06-01,122714,John & Sylvia,We had a most wonderful stay with Adriano and ...,en
4,11551,58352,2010-06-28,111543,Monique,I'm not sure which of us misunderstood the s...,en
...,...,...,...,...,...,...,...
896,36660,12713011,2014-05-09,2289602,Therese,So glad that we chose Blenheim Lodge for our 3...,en
897,36660,12940838,2014-05-15,7752,Randy,This is my 4th stay with Agri and Roger at the...,en
898,36660,12979638,2014-05-16,11142539,Bronwyn,have not time - have to arrange another booki...,en
899,36660,13464335,2014-05-28,9906875,Justyna,Cosy yet spacious room with flat screen tv and...,en


In [16]:
nltk.download('wordnet') #download corpus for lemmatizer to compare results with

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\갓민재\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
lem = WordNetLemmatizer() #create lemmatizer

In [18]:
table = str.maketrans('', '', string.punctuation) #mapping to strip punctuation in review

#strip punct of each review -> lemmatise -> output is list of words so join into sentences
lem_comments = reviews.comments.apply(lambda review: ' '.join(map(str, [lem.lemmatize(word.translate(table), pos='n') for word in review.lower().split()])))

count = lem_comments.str.count(pattern) # # of dictionary words found in each review
length = lem_comments.str.len() # review length

reviews['dict_proportion'] = count/length # save proportions

In [19]:
"Mean dictionary presence: {:.4%}".format(np.mean(reviews['dict_proportion']))

'Mean dictionary presence: 5.0550%'

In [20]:
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments,langs,dict_proportion
0,11551,30672,2010-03-21,93896,Shar-Lyn,"The flat was bright, comfortable and clean and...",en,0.079545
1,11551,32236,2010-03-29,97890,Zane,We stayed with Adriano and Valerio for a week ...,en,0.045643
2,11551,41044,2010-05-09,104133,Chase,Adriano was a fantastic host. We felt very at ...,en,0.059322
3,11551,48926,2010-06-01,122714,John & Sylvia,We had a most wonderful stay with Adriano and ...,en,0.048338
4,11551,58352,2010-06-28,111543,Monique,I'm not sure which of us misunderstood the s...,en,0.041026
...,...,...,...,...,...,...,...,...
896,36660,12713011,2014-05-09,2289602,Therese,So glad that we chose Blenheim Lodge for our 3...,en,0.043590
897,36660,12940838,2014-05-15,7752,Randy,This is my 4th stay with Agri and Roger at the...,en,0.038655
898,36660,12979638,2014-05-16,11142539,Bronwyn,have not time - have to arrange another booki...,en,0.011628
899,36660,13464335,2014-05-28,9906875,Justyna,Cosy yet spacious room with flat screen tv and...,en,0.073394


In [21]:
#overwrite csv in en_reviews
reviews.to_csv(filepath, index = False)